<a href="https://colab.research.google.com/github/GreihMurray/NLP-2/blob/master/celticMUT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('train.tsv',sep='\t',header=None)
data.columns = ['words','pos']

In [3]:
data=data.astype(str)

In [4]:
lengths = data["words"].str.len()
argmax = np.where(lengths == lengths.max())[0]
data.iloc[argmax]

,words,pos
2286166,\tN\nní\tN\ndóigh\tN\nliom\tN\ngo\tN\nfuil\tU\...,N


In [5]:
len(data['words'][2286166])

57291

In [6]:
def bigram(df):
  gram = []
  for i in range(0,len(df['words'])-1):
    gram.append(df['words'][i]+' '+df['words'][i+1])
  return gram


In [7]:
k = bigram(data)

In [8]:
l = data['pos'][1:].tolist()

In [9]:
# BIGRAM TUPLE LIST



lis=[]
for i in range(len(k)):
   lis.append((k[i],l[i]))
  


In [10]:
# UNIGRAM TUPLE LIST

m = data['words'].tolist()
n = data['pos'].tolist()
liss=[]
for i in range(len(m)):
   liss.append((m[i],n[i]))
  


In [11]:
test_set = liss[4551353:]
train_set = liss[:4551353] 

In [12]:
  # j = df['pos'][1:].tolist()
  # b_gram = [ x + y for x, y in zip(gram, j)]

In [13]:
# data = data.groupby((np.repeat(np.arange(len(data)), 2)[:len(data)])).agg(' '.join)
# data['pos'] = data['pos'].str[2:]
# data['pos'][2528529] = 'N'

In [14]:
(data['pos']=='N').sum()

4339849

In [15]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_set}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_set}
print(len(vocab))

5
{'N', 'H', 'S', 'T', 'U'}
118706


In [16]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_set):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word[0]]

#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

In [17]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_set):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [18]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
print(tags_matrix)

[[8.4304070e-01 9.1096396e-03 1.0574059e-01 4.0774336e-03 3.8031381e-02]
 [9.5785296e-01 2.3825825e-03 3.8696427e-02 1.3693003e-04 9.3112420e-04]
 [9.4543523e-01 1.5584147e-03 5.1788010e-02 6.0805196e-05 1.1575507e-03]
 [9.3869370e-01 2.9432024e-03 5.8112592e-02 1.2524266e-04 1.2524266e-04]
 [9.6020377e-01 7.3733955e-04 3.8334955e-02 6.7030865e-05 6.5690250e-04]]


In [19]:
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,N,H,S,T,U
N,0.843041,0.009110,0.105741,0.004077,0.038031
H,0.957853,0.002383,0.038696,0.000137,0.000931
S,0.945435,0.001558,0.051788,0.000061,0.001158
T,0.938694,0.002943,0.058113,0.000125,0.000125
U,0.960204,0.000737,0.038335,0.000067,0.000657


In [53]:
def Viterbi(words, train_bag = train_set):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    i = 0
    right = 0

    start = time.time()

    for key, word in enumerate(words):
        i += 1
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['N', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)

        if state_max == word[1]:
            right += 1

        cur = time.time()
        cur_hours = divmod((cur - start), 3600)
        cur_time_list = divmod((cur - start), 60)

        print('\rAccuracy on iteration ', i, ': ', ((right/i) * 100), '%', ' Elapsed Time (HH:MM:SS): ', int(cur_hours[0]), ':', int(cur_time_list[0]), ':', int(cur_time_list[1]), sep='', end='')
        
    return list(zip(words, state))

In [21]:
# import random
# # Let's test our Viterbi algorithm on a few sample sentences of test dataset
# random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# # choose random 10 numbers
# rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# # list of 10 sents on which we test the model
# test_run = [test_set[i] for i in rndom]
 
# # list of tagged words
# test_run_base = [tup for sent in test_run for tup in sent]
 
# # list of untagged words
# test_set = [tup[0] for sent in test_run for tup in sent]a

In [22]:
test_set = train_set[4551000:]
test_run_base = [tup for tup in test_set]
print(test_set[:5])

[('go', 'N'), ('leanfaidh', 'N'), ('siadsan', 'N'), ('ar', 'N'), ('aghaidh', 'N')]


In [ ]:
import time
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_set)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i[1] == j[1]] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Accuracy on iteration 86: 91.86046511627907% Elapsed Time (HH:MM:SS): 0:4:11